In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Prepare libraries
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import sklearn
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS 
import plotly.express as px

In [ ]:
# Read in the processed file
base_url = '/content/drive/My Drive/550_project/'
df_all = pd.read_csv(base_url + "df_all.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.



In [ ]:
df_all.shape

(119987, 37)

In [ ]:
df_all[pd.isnull(df_all['disaster_emergency_fund_codes_for_overall_award'])]

,assistance_transaction_unique_key,award_id_fain,federal_action_obligation,total_obligated_amount,non_federal_funding_amount,total_non_federal_funding_amount,disaster_emergency_fund_codes_for_overall_award,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_code,awarding_agency_name,funding_agency_code,funding_agency_name,object_classes_funding_this_award,program_activities_funding_this_award,recipient_duns,recipient_name,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_country_name,recipient_city_code,recipient_city_name,recipient_state_code,recipient_state_name,recipient_zip_code,primary_place_of_performance_scope,cfda_number,cfda_title,assistance_type_code,assistance_type_description,award_description,action_type_code,action_type_description,obligated_amount_for_covid19


In [ ]:
pd.isna(df_all['recipient_parent_duns']).unique()

array([False])

In [ ]:
print(df_all.columns)

Index(['assistance_transaction_unique_key', 'award_id_fain',
       'federal_action_obligation', 'total_obligated_amount',
       'non_federal_funding_amount', 'total_non_federal_funding_amount',
       'disaster_emergency_fund_codes_for_overall_award', 'action_date',
       'action_date_fiscal_year', 'period_of_performance_start_date',
       'period_of_performance_current_end_date', 'awarding_agency_code',
       'awarding_agency_name', 'funding_agency_code', 'funding_agency_name',
       'object_classes_funding_this_award',
       'program_activities_funding_this_award', 'recipient_duns',
       'recipient_name', 'recipient_parent_duns', 'recipient_parent_name',
       'recipient_country_code', 'recipient_country_name',
       'recipient_city_code', 'recipient_city_name', 'recipient_state_code',
       'recipient_state_name', 'recipient_zip_code',
       'primary_place_of_performance_scope', 'cfda_number', 'cfda_title',
       'assistance_type_code', 'assistance_type_description',
 

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162163 entries, 0 to 162162
Data columns (total 37 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   assistance_transaction_unique_key                162163 non-null  object 
 1   award_id_fain                                    162163 non-null  object 
 2   federal_action_obligation                        162163 non-null  float64
 3   total_obligated_amount                           162163 non-null  float64
 4   non_federal_funding_amount                       162163 non-null  float64
 5   total_non_federal_funding_amount                 162163 non-null  float64
 6   disaster_emergency_fund_codes_for_overall_award  48182 non-null   object 
 7   action_date                                      162163 non-null  object 
 8   action_date_fiscal_year                          162163 non-null  int64  
 9   period_of_perfo

# Create Tables

#### Agency

In [ ]:
### Agency
# Get agency-related columns
award_cols = ['awarding_agency_code', 'awarding_agency_name']
funding_cols = ['funding_agency_code', 'funding_agency_name']
award = df_all[award_cols]
funding = df_all[funding_cols]

award_ag = award.groupby(by='awarding_agency_code').max().reset_index()
award_ag.columns = ['agency_code', 'agency_name']
funding_ag = funding.groupby(by='funding_agency_code').max().reset_index()
funding_ag.columns = ['agency_code', 'agency_name']

In [ ]:
agency = pd.concat([award_ag, funding_ag], axis=0) # concatenate funding/awarding agency (just in case each table contains a different val.
# It doesn't as proven)

In [ ]:
agency.shape

(64, 2)

In [ ]:
agency_final = agency.groupby(by='agency_code').max().reset_index()
agency_final.shape

(31, 2)

In [ ]:
agency_final.to_csv(base_url + "agency.csv", index=False)

In [ ]:
agency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 31
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   agency_code  64 non-null     int64 
 1   agency_name  64 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [ ]:
award_ag.shape

(32, 2)

In [ ]:
funding_ag.shape

(32, 2)

#### Award (is it necessary?)

In [ ]:
award_type_cols = [ 'award_id_fain', 'disaster_emergency_fund_codes_for_overall_award', 
                   'object_classes_funding_this_award', 'program_activities_funding_this_award',
                   'recipient_duns', 'award_description', 'primary_place_of_performance_scope']
awards = df_all[award_type_cols]

In [ ]:
x = [ 'award_id_fain', 'primary_place_of_performance_scope']
test = awards[x].groupby(by=['award_id_fain']).max().reset_index()

In [ ]:
test.head()

,award_id_fain,primary_place_of_performance_scope
0,0000000000694345,County-wide
1,0000000000778888,County-wide
2,0000000000797432,County-wide
3,0000000000813878,County-wide
4,0000000000814594,County-wide


In [ ]:
award_type_cols_partial = [ 'award_id_fain', 'disaster_emergency_fund_codes_for_overall_award', 
                   'object_classes_funding_this_award', 'program_activities_funding_this_award',
                   'recipient_duns', 'primary_place_of_performance_scope']
awards_final = awards.groupby(by=award_type_cols_partial).max().reset_index()

In [ ]:
awards_final.head()

In [ ]:
# Remember: award came from the beast table, which can have multiple rows for the same award. So it's natural
# for "awards" table to be bigger than "awards_final."
awards_final.shape

(70264, 7)

In [ ]:
len(awards['award_id_fain'].unique())

66061

In [ ]:
len(awards_final['award_id_fain'].unique())

66061

In [ ]:
awards_final.to_csv(base_url + "award.csv", index=False)

#### Transaction (1 trans belongs to 1 award; 1 award can have many trans)

In [ ]:
trans_cols = [ 'assistance_transaction_unique_key', 'award_id_fain', 'federal_action_obligation', 'total_obligated_amount',
       'non_federal_funding_amount', 'total_non_federal_funding_amount', 'action_date',
       'action_date_fiscal_year', 'period_of_performance_start_date',
       'period_of_performance_current_end_date', 'awarding_agency_code',
       'funding_agency_code', 'obligated_amount_for_covid19', 'cfda_number',
       'assistance_type_code', 'action_type_code']
trans_final = df_all[trans_cols]

In [ ]:
trans_final.shape

(162163, 16)

In [ ]:
trans_final.to_csv(base_url + "transaction.csv", index=False)

#### Recipient

In [ ]:
recip_cols = [ 'recipient_duns', 'recipient_name', 'recipient_parent_duns', 'recipient_parent_name',
              'recipient_country_code', 'recipient_city_code', 'recipient_state_code', 'recipient_zip_code']
recip = df_all[recip_cols]

In [ ]:
recip_cols_partial = [ 'recipient_duns', 'recipient_name', 'recipient_parent_duns', 'recipient_parent_name',
              'recipient_country_code', 'recipient_city_code', 'recipient_state_code']
recip_final = recip.groupby(by=recip_cols_partial).max().reset_index()

In [ ]:
len(recip_final['recipient_duns'].unique())

18705

In [ ]:
len(recip['recipient_duns'].unique())

18705

In [ ]:
recip_final.head()

,recipient_duns,recipient_name,recipient_parent_duns,recipient_parent_name,recipient_country_code,recipient_city_code,recipient_state_code,recipient_zip_code
0,1.0,BUCKEYE COMMUNITY,None,None,USA,18000,OH,43231.0
1,1004944.0,"HAMBURG, TOWN OF",1004944.0,"HAMBURG, TOWN OF",USA,31665.0,NY,14075.0
2,1069921.0,COPLEY FURNITURE COMPANY INC,None,None,USA,77710.0,MA,2132.0
3,1118574.0,NORTHERN CHEYENNE TRIBE,70744123.0,None,USA,42250,MT,59043.0
4,1118574.0,NORTHERN CHEYENNE TRIBE,70744123.0,None,USA,42250.0,MT,59043.0


In [ ]:
recip_final.to_csv(base_url + "recipient.csv", index=False)

In [ ]:
len(recip['recipient_duns'].unique())

23074

In [ ]:
recip.shape
# What could have caused this discrepancy in recip.shape and recip_final.shape? Maybe it's because we have mixed type like "31665" and "31665.0"
# in the original recip file (note that we got it from the transactions table, so it's natural to have duplicates) 

# Reason that recip_final has fewer values: NAN groups (rows that have nan values somewhere) are automatically excluded in groupby. 
# Some of the "parent_duns" have Nans.

# Real problem (resolved): when dropna() in the other file, didn't drop some na's, so the na's get ignored in groupby, causing inconsistency

(162163, 8)

#### City

In [ ]:
city_col = [ 'recipient_city_code', 'recipient_city_name']
city = df_all[city_col]

In [ ]:
city_final = city.groupby(by=['recipient_city_code']).max().reset_index()

In [ ]:
len(city_final['recipient_city_code'].unique())

11084

In [ ]:
len(city['recipient_city_code'].unique())

11084

In [ ]:
city_final.head()

,recipient_city_code,recipient_city_name
0,100,ABERDEEN
1,148,ABINGDON
2,200,ADA
3,358,ADJUNTAS
4,460,AITKIN


In [ ]:
city_final.to_csv(base_url + "city.csv", index=False)

#### State

In [ ]:
state_cols = [ 'recipient_state_code', 'recipient_state_name']
state = df_all[state_cols]

In [ ]:
state_final = state.groupby(by=['recipient_state_code']).max().reset_index()

In [ ]:
len(state_final['recipient_state_code'].unique())

56

In [ ]:
len(state['recipient_state_code'].unique())

56

In [ ]:
state_final.head()

,recipient_state_code,recipient_state_name
0,AK,ALASKA
1,AL,ALABAMA
2,AR,ARKANSAS
3,AS,AMERICAN SAMOA
4,AZ,ARIZONA


In [ ]:
state_final.shape

(56, 2)

In [ ]:
state_final.to_csv(base_url + "state.csv", index=False)

#### Area_of_work

In [ ]:
aow_cols = [ 'cfda_number', 'cfda_title' ]
aow = df_all[aow_cols]

In [ ]:
aow_final = aow.groupby(by=['cfda_number']).max().reset_index()
aow_final.head()

,cfda_number,cfda_title
0,10.001,AGRICULTURAL RESEARCH BASIC AND APPLIED RESEARCH
1,10.025,"PLANT AND ANIMAL DISEASE, PEST CONTROL, AND AN..."
2,10.028,WILDLIFE SERVICES
3,10.030,INDEMNITY PROGRAM
4,10.124,FARM-TO-FLEET FEEDSTOCK PROGRAM BIOFUEL PRODUC...


In [ ]:
len(aow_final['cfda_number'].unique())

1104

In [ ]:
len(aow['cfda_number'].unique())

1104

In [ ]:
aow_final.to_csv(base_url + "area_of_work.csv", index=False)

#### Assistance_type

In [ ]:
assist_cols = [ 'assistance_type_code', 'assistance_type_description' ]
assist = df_all[assist_cols]

In [ ]:
assist_final = assist.groupby(by=['assistance_type_code']).max().reset_index()
assist_final.head()

,assistance_type_code,assistance_type_description
0,2,BLOCK GRANT (A)
1,3,FORMULA GRANT (A)
2,4,PROJECT GRANT (B)
3,5,COOPERATIVE AGREEMENT (B)
4,6,"DIRECT PAYMENT FOR SPECIFIED USE, AS A SUBSIDY..."


In [ ]:
len(assist_final['assistance_type_code'].unique())

8

In [ ]:
len(assist['assistance_type_code'].unique())

8

In [ ]:
assist_final.to_csv(base_url + "assistance.csv", index=False)

#### Action

In [ ]:
action_cols = [ 'action_type_code', 'action_type_description' ]
action = df_all[action_cols]

In [ ]:
action_final = action.groupby(by=['action_type_code']).max().reset_index()
action_final.head()

,action_type_code,action_type_description
0,A,NEW
1,B,CONTINUATION
2,C,REVISION
3,D,ADJUSTMENT TO COMPLETED PROJECT


In [ ]:
len(action['action_type_code'].unique())

4

In [ ]:
len(action_final['action_type_code'].unique())

4

In [ ]:
action_final.to_csv(base_url + "action.csv", index=False)